# Generalized Sequential Pattern (GSP) Mining

Here we've defined some drivers to help implementing the GSP algorithm from scratch (with some simplifying assumptions, like not having an itemset of length > 1 withing a sequence, although it matches many real situations, like text mining and website clicks analyzing).

### BEFORE START, READ THIS:
*This code is credit to WASEEM KNTAR and it's for all BUT if you have an assignment anywhere about this algorithm I prefer reading the code first and understanding it (it's not hard) THEN you can benefit from it as you want*.

In [1]:
DB = [["A", "C", "D"],
      ["A", "B", "E", "F", "C"],
      ["A", "C", "E", "C"],
      ["A", "C", "A"]]

In [2]:
from longest_common_subseq import lcs
def within(seq1, seq2):
    # returns true if seq1 is contained within seq2
    # ex.1: within([A, B, A], [X, A, A, B, X, A]) = true
    # ex.2: within([A, B, A], [X, A, A, B, X]) = false
    return lcs(seq1, seq2)[0] == len(seq1)
s1, s2 = ['A', 'B', 'A'], ['X', 'A', 'A', 'B', 'X', 'A']
print(within(s1, s2))

True


In [3]:
from dic_sort import dic_sort
def getL1(DB, min_sup = 2):
    # returns L1 vector, which contains all DB items with sup >= min_sup
    # ex: [[A, B, C], 
    #      [B],
    #      [E, K]]
    # => [[B]]
    # our use case: [[A], [C], [E]]
    vocabulary = []
    for i in range(len(DB)):
        for j in range(len(DB[i])):
            if DB[i][j] not in vocabulary:
                vocabulary.append(DB[i][j])
    dic = {}
    for i in range(len(DB)):
        for j in range(len(vocabulary)):
            if vocabulary[j] in DB[i]:
                if dic.get(vocabulary[j]) == None:
                    dic.update({vocabulary[j]: 0})
                dic[vocabulary[j]] += 1
    dic_L1 = {}
    L1 = []
    for key in dic.keys():
        if dic[key] >= min_sup:
            dic_L1.update({key: dic[key]})
            L1.append(key)
    return L1, dic_sort(dic_L1, reverse = True)

L1, dic_L1 = getL1(DB, min_sup = 2)
print(L1)

['A', 'C', 'E']


In [4]:
from permutation_and_relation import two_list_permutations
#['A', 'C', 'E']
def getL2(DB, L1, min_sup = 2):
    # returns L2 vector, which contains all length-2 itemsets with sup >= min_sup
    # ex: [A, B] => [[A, A],[A, B],[B, A],[B, B]]
    # our use case: [[A, C], [A, E], [E, C]]
    permutations = two_list_permutations(L1, L1)
    dic = {}
    #calculate the support of generated length-2 itemset
    for perm in permutations:
        for transaction in DB:
            if within(perm, transaction):
                if dic.get(perm) == None:
                    dic.update({perm: 0})
                dic[perm] += 1
    #print(dic)
    dic_L2 = {}
    L2 = []
    for key in dic.keys():
        if dic[key] >= min_sup:
            dic_L2.update({key: dic[key]})
            L2.append(list(key))
    if L2 == []:
        return None, None
    return L2, dic_sort(dic_L2, reverse = True)
L2, dic_L2 = getL2(DB, L1, min_sup = 2)
print(L2)

(False, None, None)
[['A', 'E'], ['E', 'C'], ['A', 'C']]


In [5]:
from permutation_and_relation import is_there_relation
def get_Lj_plus_1(DB, L_j, min_sup = 2):
    # returns Lk vecotr, which contains all length-(k) itemsets generated from length-(k-1) vector with sup >= min_sup
    # ex: [A, B, C] * [B, C, D] => [A, B, C, D]
    # ex: [A, B, C, D] * [B, C, D, E] => [A, B, C, D, E]
    # our use case: [A, E, C]
    # remember the method we've used in class, refer to research "Mining Sequential Patterns: Generalizations and Performance Improvements"
    # by Ramakrishnan and Rakesh Agrawal.
    results = []
    for i in range(len(L_j)):
        for j in range(i, len(L_j)):
            #print(i, j)
            rel = is_there_relation(''.join(L_j[i]), ''.join(L_j[j]))
            #print(rel)
            if rel[0]:
                if rel[1] != None:
                    results.append(rel[1])
                if rel[2] != None:
                    results.append(rel[2])
    if results == []:
        return None, None
    dic = {}
    for result in results:
        for transaction in DB:
            if within(result, transaction):
                if dic.get(result) == None:
                    dic.update({result: 0})
                dic[result] += 1
                
    dic_L_new = {}
    L_new = []
    for key in dic.keys():
        if dic[key] >= min_sup:
            dic_L_new.update({key: dic[key]})
            L_new.append(list(key))
    return L_new, dic_sort(dic_L_new, reverse = True)               
L3, dic_L3 = get_Lj_plus_1(DB, L2, min_sup = 2)
print(L3)

[['A', 'E', 'C']]


In [6]:
def GSP(DB, min_sup = 2):
    # use the above methods to get all of the frequent sequences of all lengths.
    # the only trick is to know when to break the while loop..
    # our use case:
    #  lenght-1           length-2           length-3
    #   A                  [A, C]              [A, E, C]
    #   C                  [A, E]
    #   E                  [E, C]
    Ls = []
    L1, _ = getL1(DB, min_sup)
    Ls.append(L1)
    if len(L1) >= 1:
        L2, _ = getL2(DB, L1, min_sup)
        if L2 != None:
            Ls.append(L2)
            i = 1
            while len(Ls[i]) >= 1:
                L_new, _ = get_Lj_plus_1(DB, Ls[i], min_sup)
                if L_new == None:
                    break
                Ls.append(L_new)
                i += 1
    return Ls
GSP(DB, min_sup = 2)

[['A', 'C', 'E'], [['A', 'E'], ['E', 'C'], ['A', 'C']], [['A', 'E', 'C']]]

## Let's test our implementation:

In [7]:
A = "A";
B =  "B";
C = "C";
D = "D";
E = "E";
F = "F";
X = "X";
DB = [[A, C, D],
      [A, B, E, F, C],
      [A, C, E, C],
      [A, C, A]]

In [8]:
print (within([A, B, A],[A, B]))  # false
print (within([A, B],[B, X, A])) # false
print (within([A, B, A],[A, A, X, B, X, A]))  # true
print (within([A, B],[A, X, B]))  # true

False
False
True
True


In [9]:
print (getL1([[A, A]], min_sup = 2)[0]) # []
print (getL1([[A, A]], min_sup = 1)[0]) # [A]
print (getL1([[A, B], [C]], min_sup = 2)[0]) # []
print (getL1([[A, B], [C]], min_sup = 1)[0]) # [[A], [B], [C]]
print (getL1([[A, A], [A]], min_sup = 2)[0]) # [A]

[]
['A']
[]
['A', 'B', 'C']
['A']


In [10]:
DB = [[A, C, D],
      [A, B, E, F, C],
      [A, C, E, C],
      [A, C, A]]
print (getL2(DB, [A, B], min_sup = 1)[0])# [[A, B], [A, A]]
print (getL2(DB, [A, B], min_sup = 2)[0])# [] or [[]]
print (getL2(DB, [A, C], min_sup = 1)[0])  # [[A, C], [C, A], [A, A], [C, C]]
print (getL2(DB, [A, B, F], min_sup = 2)[0]) # []
print (getL2(DB, [A, B, F], min_sup = 1)[0]) # [[A, B], [A, F], [B, F], [A, A]]

[['A', 'B'], ['A', 'A']]
None
[['A', 'C'], ['C', 'A'], ['C', 'C'], ['A', 'A']]
None
[['B', 'F'], ['A', 'A'], ['A', 'B'], ['A', 'F']]


In [11]:
DB = [[A, C, D],
      [A, B, E, F, C],
      [A, C, E, C],
      [A, C, A]]
print (GSP(DB, min_sup = 2)) # L1 = [[A], [C], [E]], L2 = [[A, C], [A, E], [E, C]], L3 = [[A, E, C]]

[['A', 'C', 'E'], [['A', 'E'], ['E', 'C'], ['A', 'C']], [['A', 'E', 'C']]]


In [12]:
DB = [[A, C, D],
      [A, B, E, F, C],
      [A, C, E, C],
      [A, C, A]]
print (GSP(DB, min_sup = 3)) # L1 = [[A], [C]], L2 = [[A, C]]

[['A', 'C'], [['A', 'C']]]


In [13]:
DB2 = [[A, C, D, A], # HERE WE ADD {A} to the end of the sequence
      [A, B, E, F, C],
      [A, C, E, C],
      [A, C, A]]
print (GSP(DB2, min_sup = 2)) # L1 = [[A], [C], [E]], L2 = [[A, E], [E, C], [C, A], [A, A]], L3 = [[A, C, A], [A, E, C]]

[['A', 'C', 'E'], [['C', 'A'], ['A', 'E'], ['E', 'C'], ['A', 'A'], ['A', 'C']], [['A', 'C', 'A'], ['A', 'E', 'C']], []]


In [14]:
DB3 = [[A, A], # HERE WE ADD {A} to the end of the sequence
      [A],
      [A, A, A]]
print (GSP(DB3, min_sup = 2)) # L1 = [[A]], L2 = [[A, A]]

[['A'], [['A', 'A']], []]


In [15]:
DB3 = [[A, A], # HERE WE ADD {A} to the end of the sequence
      [A],
      [A, A, A]]
print (GSP(DB3, min_sup = 1)) # L1 = [[A]], L2 = [[A, A]], L3 = [[A, A, A]]

[['A'], [['A', 'A']], [['A', 'A', 'A']], []]
